# Run Objective:

### Run0145-karel_edition-Mk3


Replaced

```power_layer = Lambda(lambda x: (K.clip(K.abs(x[0]), 0.0, 10)) ** (K.clip(K.abs(x[1]), 0.0, 2.0)))```

with

```power_layer = Lambda(lambda x: (K.clip(K.abs(x[0]), 0.0, 10)) ** (K.clip(K.abs(x[1]), 1.0, 2.0)))```


-  Karel didn't connect up the `m_branch` in `Run0145-karel_edition-Mk2`. This has now been added back in.

In [ ]:
# DO NOT RUN THIS PART

In [ ]:
import os
import sys
import time

# Gets the current file name. Useful for procedurally generating output/log files.
file_name =  os.path.basename(sys.argv[0][:-3])
print(file_name)

if file_name == "ipykernel_launcher":
    print("This is the Jupyter version.")
    print("Now MANUALLY run the next two cells!")
    print("STOP! This should not be in your code!!")
    exit(0)
    time.sleep(10)
    print("Testing if script has really stopped.")
else:
    print("This is the Atom version")

In [1]:
# RUN ONLY IN JUPYTER!!
# Start here (manual)

In [2]:
%%javascript
IPython.notebook.kernel.execute('file_name = "' + IPython.notebook.notebook_name + '"');

<IPython.core.display.Javascript object>

In [3]:
print(file_name)

2018-11-08_Run0145-karel_edition-Mk3.ipynb


In [4]:
file_name = file_name[:-6]
print(file_name)

is_Jupyter = True
print(is_Jupyter)

2018-11-08_Run0145-karel_edition-Mk3
True


In [5]:
# Same code for both ATOM & JUPYTER from now (Run all cells below now)

In [6]:
'''
Late Fusion Module (test) - Functional API
'''

# Multiple Inputs
import keras
from keras.optimizers import RMSprop, adam, Adam
from keras.initializers import TruncatedNormal, glorot_normal, Constant
from keras.utils import plot_model
from keras.models import Model
from keras.layers.core import Lambda
from keras.layers import Input
from keras.layers import Dense
from keras.layers import MaxoutDense
from keras.layers.merge import concatenate
from keras import regularizers
from keras import backend as K
#from keras.backend import switch
import pandas
import numpy
import sys
import os
from copy import deepcopy
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.layers.advanced_activations import ThresholdedReLU

#keras.backend.clear_session()
K.clear_session()

Using TensorFlow backend.


In [7]:
# Define new Metric: rmse = Root Mean Square Error
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square( y_true-y_pred )))

In [8]:
# Modify for ATOM use

In [9]:
if is_Jupyter == True:
    pass
else:
    # Gets the current file name. Useful for procedurally generating output/log files.
    file_name =  os.path.basename(sys.argv[0][:-3])

In [10]:
# Define neural network parameters
batch_size = 10
#num_classes = 1
epochs = 100

In [11]:
# Load Data (which is in HDF5 or .h5 format)
store = pandas.HDFStore("training_gen3_7D_nions0_flat_filter8.h5")
target_df = store['/output/efeETG_GB'].to_frame()  # This one is relatively easy to train
input_df = store['input']

# Puts inputs and outputs in the same pandas dataframe.
# Also only keeps overlapping entries.
joined_dataFrame = target_df.join(input_df)

# Make a copy of joined_dataFrame for later use
joined_dataFrame_original = deepcopy(joined_dataFrame)


# *************************************************************************** #
# Normalize data by standard deviation and mean-centering the data
# Standard configuration
joined_dataFrame['efeETG_GB'] = (joined_dataFrame['efeETG_GB'] - joined_dataFrame['efeETG_GB'].mean()) / joined_dataFrame['efeETG_GB'].std()
joined_dataFrame['Ati'] = (joined_dataFrame['Ati'] - joined_dataFrame['Ati'].mean()) / joined_dataFrame['Ati'].std()
joined_dataFrame['Ate'] = (joined_dataFrame['Ate'] - joined_dataFrame['Ate'].mean()) / joined_dataFrame['Ate'].std()
joined_dataFrame['An'] = (joined_dataFrame['An'] - joined_dataFrame['An'].mean()) / joined_dataFrame['An'].std()
joined_dataFrame['q'] = (joined_dataFrame['q'] - joined_dataFrame['q'].mean()) / joined_dataFrame['q'].std()
joined_dataFrame['smag'] = (joined_dataFrame['smag'] - joined_dataFrame['smag'].mean()) / joined_dataFrame['smag'].std()
joined_dataFrame['x'] = (joined_dataFrame['x'] - joined_dataFrame['x'].mean()) / joined_dataFrame['x'].std()
joined_dataFrame['Ti_Te'] = (joined_dataFrame['Ti_Te'] - joined_dataFrame['Ti_Te'].mean()) / joined_dataFrame['Ti_Te'].std()

# Shuffles dataset
shuffled_joined_dataFrame = joined_dataFrame.reindex(numpy.random.permutation(
                                                joined_dataFrame.index))

# Creates a pandas dataframe for the outputs
shuffled_clean_output_df = shuffled_joined_dataFrame['efeETG_GB']

# Make a copy of shuffled_joined_dataFrame for later use
shuffled_joined_dataFrame_base = deepcopy(shuffled_joined_dataFrame)



# *************************************************************************** #
# Creates a pandas dataframe for the inputs (7D)
shuffled_clean_input_df_7D = shuffled_joined_dataFrame.drop('efeETG_GB', axis=1)

# Creates training dataset (90% of total data) for outputs
y_train = shuffled_clean_output_df.iloc[:int(
    numpy.round(len(shuffled_clean_output_df)*0.9))]

# Creates training dataset (90% of total data) for inputs
x_train = shuffled_clean_input_df_7D.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_7D)*0.9))]

# Creates testing dataset (10% of total data) for outputs
y_test = shuffled_clean_output_df.iloc[int(
    numpy.round(len(shuffled_clean_output_df)*0.9)):]

# Creates testing dataset (10% of total data) for inputs
x_test = shuffled_clean_input_df_7D.iloc[int(
    numpy.round(len(shuffled_clean_input_df_7D)*0.9)):]
# *************************************************************************** #


# Deletes pandas dataframes that are no longer needed
del target_df, input_df

# Closes the HDFStore. This is good practice.
store.close()


In [12]:
shuffled_clean_input_df_7D.describe(include='all')

,Ati,Ate,An,q,smag,x,Ti_Te
count,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06
mean,-1.408126e-16,2.629128e-16,2.264543e-16,8.714849e-17,-1.771987e-17,-6.755442e-19,5.202348e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.632300e+00,-1.639065e+00,-1.901001e+00,-8.370116e-01,-1.574307e+00,-1.405340e+00,-1.235044e+00
25%,-7.928000e-01,-7.898708e-01,-5.438138e-01,-6.367600e-01,-8.011463e-01,-9.907529e-01,-8.817246e-01
50%,-1.059360e-01,-9.507533e-02,1.347799e-01,-3.983652e-01,-1.936629e-01,-1.615792e-01,-1.750863e-01
75%,5.809279e-01,5.997202e-01,6.437252e-01,1.976217e-01,9.108525e-01,6.675945e-01,7.576761e-01
max,2.641520e+00,2.684107e+00,1.831264e+00,2.581570e+00,1.739239e+00,1.704062e+00,1.944828e+00


In [13]:
# Creates a pandas dataframe for the inputs
shuffled_clean_input_df_1 = shuffled_clean_input_df_7D.drop('Ate', axis=1)

In [14]:
shuffled_clean_input_df_1.describe(include='all')

,Ati,An,q,smag,x,Ti_Te
count,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06,6.479137e+06
mean,-1.408126e-16,2.264543e-16,8.714849e-17,-1.771987e-17,-6.755442e-19,5.202348e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.632300e+00,-1.901001e+00,-8.370116e-01,-1.574307e+00,-1.405340e+00,-1.235044e+00
25%,-7.928000e-01,-5.438138e-01,-6.367600e-01,-8.011463e-01,-9.907529e-01,-8.817246e-01
50%,-1.059360e-01,1.347799e-01,-3.983652e-01,-1.936629e-01,-1.615792e-01,-1.750863e-01
75%,5.809279e-01,6.437252e-01,1.976217e-01,9.108525e-01,6.675945e-01,7.576761e-01
max,2.641520e+00,1.831264e+00,2.581570e+00,1.739239e+00,1.704062e+00,1.944828e+00


In [15]:
print(shuffled_clean_input_df_1.shape)

(6479137, 6)


In [16]:
shuffled_clean_input_df_2 = shuffled_clean_input_df_7D.drop('Ati', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('An', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('q', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('smag', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('x', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('Ti_Te', axis=1)

In [17]:
shuffled_clean_input_df_2.describe(include='all')

,Ate
count,6.479137e+06
mean,2.629128e-16
std,1.000000e+00
min,-1.639065e+00
25%,-7.898708e-01
50%,-9.507533e-02
75%,5.997202e-01
max,2.684107e+00


In [18]:
print(shuffled_clean_input_df_2.shape)

(6479137, 1)


In [19]:
# *************************************************************************** #
# Branch 1

# Creates training dataset (90% of total data) for inputs
x_train_1 = shuffled_clean_input_df_1.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_1)*0.9))]

# Creates testing dataset (10% of total data) for inputs
x_test_1 = shuffled_clean_input_df_1.iloc[int(
    numpy.round(len(shuffled_clean_input_df_1)*0.9)):]
# *************************************************************************** #

In [20]:
# *************************************************************************** #
# Branch 2

# Creates training dataset (90% of total data) for inputs
x_train_2 = shuffled_clean_input_df_2.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_2)*0.9))]

# Creates testing dataset (10% of total data) for inputs
x_test_2 = shuffled_clean_input_df_2.iloc[int(
    numpy.round(len(shuffled_clean_input_df_2)*0.9)):]
# *************************************************************************** #

In [ ]:
#model.layers[7].get_weights()

In [ ]:
#model.layers[3].get_weights()

In [ ]:
#model.get_layer("c_3_branch3_feeder").get_weights()

In [ ]:
#model.get_layer("c_3_branch3_feeder").get_weights()[0]

In [ ]:
#model.get_layer("c_3_branch3_feeder").get_weights()[0][0]

In [ ]:
#model.get_layer("c_3_branch3_feeder").get_weights()[0][0][0]

In [ ]:
#model.layers[14] 

In [ ]:
#model.layers[14].get_weights()

In [ ]:
'''
def power_function(x):
    #from keras import backend as K
    x = model.get_layer("c_3_branch3_feeder").get_weights()[0]
    c_3 = K.cast(x, dtype='float32')
    if (c_3 >= 0.0):
        return c_3
    else:
        return 0.0
    
    return 1.0 ** c_3
'''

In [ ]:
'''
def power_function(x):
    #from keras import backend as K
    x = model.get_layer("c_3_branch3_feeder").get_weights()[0]
    c_3 = K.cast(x, dtype='float32')
    #if (c_3 >= 0.0):
    #    pass
    #else:
    #    c_3 = 0.0
    #
    return 1.0 ** c_3
'''

In [ ]:
# counter = 0

In [ ]:
'''
def power_function(x):
    print("counter = 0")
    print("x: ")
    print(x)
    print("type(x): ")
    print(type(x))
    counter = 1
    if (counter == 0):
        pass
    else:
        print("counter = 1")
        print("x: ")
        print(x)
        print("type(x): ")
        print(type(x))
        a = K.cast(x, dtype='float32')
        print("a: ")
        print(a)
        try:
            print(model.layers[14])
            c_3 = model.layers[14].get_weights()[0][0][0]
            print(c_3)
            print("type(c_3): ")
            print(type(c_3))
            if (c_3 >= 0.0):
                pass
            else:
                c_3 = 0.0
            pooling = 1.0 ** c_3
            power_pooling_output = K.cast(pooling, dtype='float32')
        except:
            print("An exception occurred")
    return power_pooling_output
'''

In [ ]:
#model.add(Lambda(lambda x: 1 ** x))

In [21]:
# branch1
visible_branch1 = Input(shape=(6, ), name="6D_INPUTS")
hidden1_branch1 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden1_branch1")(visible_branch1)
hidden2_branch1 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden2_branch1")(hidden1_branch1)
theta_branch1 = Dense(1,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="theta_branch1")(hidden2_branch1)
theta_branch1_feeder = Dense(1,
        activation='linear',
        kernel_initializer=Constant(value=-1),
        bias_initializer='Zeros',
        trainable=False,
        name="theta_branch1_feeder")(theta_branch1)

# branch2 (Ate input)
visible_branch2 = Input(shape=(1, ), name="Ate_INPUT")
visible_branch2_feeder = Dense(1,
                activation='linear',
                name="visible_branch2_feeder")(visible_branch2)

# addition_pooling (effectively subtraction though...)
addition_pooling = keras.layers.Add(name="Addition_Pooling")([theta_branch1_feeder, visible_branch2_feeder])

TR = Dense(1, activation='relu',
           kernel_initializer='Ones',
           bias_initializer='Zeros',
           trainable=False,
           name="TR")(addition_pooling)

# branch 3 (for c_3)
hidden1_branch3 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden1_branch3")(visible_branch1)
hidden2_branch3 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden2_branch3")(hidden1_branch3)
c_3_branch3 = Dense(1,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="c_3_branch3")(hidden2_branch3)
c_3_branch3_feeder = Dense(1,
        kernel_initializer=Constant(value=+1),
        bias_initializer='Zeros',
        trainable=False,
        name="c_3_branch3_feeder")(c_3_branch3)

# Power_Pooling
#merge = concatenate([TR, c_3_branch3_feeder], name="merge")
#power_pooling = Lambda(power_function, name="Power_Pooling")(merge)
#power_pooling = Lambda(lambda x: 1 ** x)(c_3_branch3_feeder)
def power_lambda(x):
    if x[0] <= 0:
        return 0
    else:
        return x[0] ** x[1]
power_layer = Lambda(lambda x: (K.clip(K.abs(x[0]), 1e-32, numpy.inf)) ** (K.clip(K.abs(x[1]), -1., 3.)))
#power_layer = Lambda(lambda x: K.tf.where(K.tf.less_equal(x[0], 1e-6), K.tf.fill(tf.shape(x[0]), 0.), K.abs(x[0]) ** K.clip(K.abs(x[1]), 0.5, 3.)))
#power_layer = Lambda(lambda x: K.tf.where(K.tf.greater(x[0], 1e-6), K.tf.fill(tf.shape(x[0]), 0.), K.tf.fill(tf.shape(x[0]), 1000000000.)))

power_pooling = power_layer([TR, c_3_branch3_feeder])


# branch 4 (for the gradient)
hidden1_branch4 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden1_branch4")(visible_branch1)
hidden2_branch4 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="hidden2_branch4")(hidden1_branch4)
m_branch4 = Dense(1,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal',
        name="m_branch4")(hidden2_branch4)
m_branch4_feeder = Dense(1,
        activation='linear',
        kernel_initializer=Constant(value=+1),
        bias_initializer='Zeros',
        trainable=False,
        name="m_branch4_feeder")(m_branch4)

# multiplication pooling
multiplication_pooling = keras.layers.Multiply(name="Multiplication_Pooling")([m_branch4_feeder, power_pooling])

# interpretation model
output = Dense(1, activation='linear',
           kernel_initializer='Ones',
           bias_initializer='Zeros',
           trainable=True,
           name="Output_Layer")(multiplication_pooling)

model = Model(inputs=[visible_branch1, visible_branch2], outputs=output)

# summarize layers
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
6D_INPUTS (InputLayer)          (None, 6)            0                                            
__________________________________________________________________________________________________
hidden1_branch1 (Dense)         (None, 30)           210         6D_INPUTS[0][0]                  
__________________________________________________________________________________________________
hidden2_branch1 (Dense)         (None, 30)           930         hidden1_branch1[0][0]            
__________________________________________________________________________________________________
theta_branch1 (Dense)           (None, 1)            31          hidden2_branch1[0][0]            
__________________________________________________________________________________________________
Ate_INPUT 

In [ ]:
# plot graph
plot_model(model, 'ModelPlots/' + str(file_name) + '_model_plot.png')

In [22]:
from IPython import embed
class NBatchLogger(keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                if abs(val) > 1e-3:
                    metrics_log += ' - %s: %.4f' % (k, val)
                else:
                    metrics_log += ' - %s: %.4e' % (k, val)
            if self.params['steps'] is None:
                steps_per_epoch = int(numpy.ceil(self.params['samples'] / self.params['batch_size']))
            else:
                steps = self.params['steps']
            #embed()
            epoch = int(numpy.floor((self.step - 1)/ steps_per_epoch))
            step_in_epoch = self.step - epoch * steps_per_epoch
            print('step: {}/{} epoch {}: ... {}'.format(step_in_epoch,
                                          steps_per_epoch,
                                          epoch,
                                          metrics_log))
            self.metric_cache.clear()

In [23]:
model.compile(loss='mean_squared_error',   #categorical_crossentropy
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=["mae", "mean_squared_error", rmse])

# Add CallBacks (including TensorBoard)
tbCallBack = keras.callbacks.TensorBoard(
        log_dir='TensorBoard_logs/' + str(file_name), write_graph = False, write_images=False, write_grads=False)
EarlyStoppingCallBack = keras.callbacks.EarlyStopping(
        monitor='val_rmse', min_delta=0, patience=15, verbose=0, mode='auto')

history = model.fit([x_train_1, x_train_2],
                    y = y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=2,
                    # validation_data=(x_test, y_test),
                    validation_data=([x_test_1, x_test_2], y_test),
                    callbacks=[tbCallBack, EarlyStoppingCallBack, NBatchLogger(5e4)])
#5e4

# score = model.evaluate(x_test, y_test, verbose=0)
score = model.evaluate([x_test_1, x_test_2], y_test, verbose=0)

print('Test loss:', score[0])
print('val_mean_absolute_error:', score[1])

print("score")
print(score)

print("model.metrics_names")
print(model.metrics_names)

# creates a HDF5 file 'my_model.h5'
model.save("./Saved-Networks/" + str(file_name) +".h5")

# Create output file
OutputFile = open("./Loss-Values/" +str(file_name) +".txt", "w+")
OutputFile.write("Test loss: " + str(score[0]) + "\n")
OutputFile.write("val_mean_absolute_error: " +str(score[1]) + "\n")
OutputFile.write("val_mean_squared_error: " +str(score[2]) + "\n")
OutputFile.write("RMSE: " +str(score[3]) + "\n")
OutputFile.close()

del history
del model


Train on 5831223 samples, validate on 647914 samples
Epoch 1/100
step: 50000/583123 epoch 0: ...  - loss: 0.5076 - mean_absolute_error: 0.3093 - mean_squared_error: 0.5017 - rmse: 0.5383
step: 100000/583123 epoch 0: ...  - loss: 0.4039 - mean_absolute_error: 0.2838 - mean_squared_error: 0.3969 - rmse: 0.4850
step: 150000/583123 epoch 0: ...  - loss: 0.3443 - mean_absolute_error: 0.2639 - mean_squared_error: 0.3366 - rmse: 0.4463
step: 200000/583123 epoch 0: ...  - loss: 0.3554 - mean_absolute_error: 0.2800 - mean_squared_error: 0.3465 - rmse: 0.4599
step: 250000/583123 epoch 0: ...  - loss: 0.3505 - mean_absolute_error: 0.2710 - mean_squared_error: 0.3415 - rmse: 0.4519
step: 300000/583123 epoch 0: ...  - loss: 0.3634 - mean_absolute_error: 0.2832 - mean_squared_error: 0.3536 - rmse: 0.4650
step: 350000/583123 epoch 0: ...  - loss: 0.3342 - mean_absolute_error: 0.2722 - mean_squared_error: 0.3243 - rmse: 0.4447
step: 400000/583123 epoch 0: ...  - loss: 0.3083 - mean_absolute_error: 0.2

Epoch 6/100
step: 34385/583123 epoch 5: ...  - loss: 0.3179 - mean_absolute_error: 0.2708 - mean_squared_error: 0.3080 - rmse: 0.4367
step: 84385/583123 epoch 5: ...  - loss: 0.3030 - mean_absolute_error: 0.2484 - mean_squared_error: 0.2934 - rmse: 0.4163
step: 134385/583123 epoch 5: ...  - loss: 0.3262 - mean_absolute_error: 0.2642 - mean_squared_error: 0.3167 - rmse: 0.4355
step: 184385/583123 epoch 5: ...  - loss: 0.3218 - mean_absolute_error: 0.2641 - mean_squared_error: 0.3124 - rmse: 0.4341
step: 234385/583123 epoch 5: ...  - loss: 0.3247 - mean_absolute_error: 0.2632 - mean_squared_error: 0.3152 - rmse: 0.4328
step: 284385/583123 epoch 5: ...  - loss: 0.3233 - mean_absolute_error: 0.2627 - mean_squared_error: 0.3133 - rmse: 0.4341
step: 334385/583123 epoch 5: ...  - loss: 0.3150 - mean_absolute_error: 0.2463 - mean_squared_error: 0.3054 - rmse: 0.4197
step: 384385/583123 epoch 5: ...  - loss: 0.3152 - mean_absolute_error: 0.2473 - mean_squared_error: 0.3058 - rmse: 0.4205
step: 

Epoch 11/100
step: 18770/583123 epoch 10: ...  - loss: 0.3306 - mean_absolute_error: 0.2685 - mean_squared_error: 0.3220 - rmse: 0.4412
step: 68770/583123 epoch 10: ...  - loss: 0.3062 - mean_absolute_error: 0.2471 - mean_squared_error: 0.2972 - rmse: 0.4170
step: 118770/583123 epoch 10: ...  - loss: 0.3026 - mean_absolute_error: 0.2469 - mean_squared_error: 0.2938 - rmse: 0.4153
step: 168770/583123 epoch 10: ...  - loss: 0.3593 - mean_absolute_error: 0.2782 - mean_squared_error: 0.3499 - rmse: 0.4574
step: 218770/583123 epoch 10: ...  - loss: 0.3192 - mean_absolute_error: 0.2594 - mean_squared_error: 0.3101 - rmse: 0.4282
step: 268770/583123 epoch 10: ...  - loss: 0.3259 - mean_absolute_error: 0.2647 - mean_squared_error: 0.3165 - rmse: 0.4356
step: 318770/583123 epoch 10: ...  - loss: 0.3284 - mean_absolute_error: 0.2575 - mean_squared_error: 0.3191 - rmse: 0.4316
step: 368770/583123 epoch 10: ...  - loss: 0.3297 - mean_absolute_error: 0.2725 - mean_squared_error: 0.3201 - rmse: 0.44

Epoch 16/100
step: 3155/583123 epoch 15: ...  - loss: 0.3149 - mean_absolute_error: 0.2426 - mean_squared_error: 0.3057 - rmse: 0.4175
step: 53155/583123 epoch 15: ...  - loss: 0.3085 - mean_absolute_error: 0.2356 - mean_squared_error: 0.2995 - rmse: 0.4105
step: 103155/583123 epoch 15: ...  - loss: 0.3130 - mean_absolute_error: 0.2487 - mean_squared_error: 0.3038 - rmse: 0.4206
step: 153155/583123 epoch 15: ...  - loss: 0.3034 - mean_absolute_error: 0.2463 - mean_squared_error: 0.2940 - rmse: 0.4143
step: 203155/583123 epoch 15: ...  - loss: 0.3021 - mean_absolute_error: 0.2417 - mean_squared_error: 0.2928 - rmse: 0.4111
step: 253155/583123 epoch 15: ...  - loss: 0.3070 - mean_absolute_error: 0.2404 - mean_squared_error: 0.2977 - rmse: 0.4117
step: 303155/583123 epoch 15: ...  - loss: 0.3230 - mean_absolute_error: 0.2497 - mean_squared_error: 0.3139 - rmse: 0.4243
step: 353155/583123 epoch 15: ...  - loss: 0.3359 - mean_absolute_error: 0.2549 - mean_squared_error: 0.3267 - rmse: 0.434

Epoch 21/100
step: 37540/583123 epoch 20: ...  - loss: 0.3104 - mean_absolute_error: 0.2410 - mean_squared_error: 0.3012 - rmse: 0.4137
step: 87540/583123 epoch 20: ...  - loss: 0.3223 - mean_absolute_error: 0.2579 - mean_squared_error: 0.3133 - rmse: 0.4307
step: 137540/583123 epoch 20: ...  - loss: 0.3566 - mean_absolute_error: 0.2686 - mean_squared_error: 0.3474 - rmse: 0.4503
step: 187540/583123 epoch 20: ...  - loss: 0.3147 - mean_absolute_error: 0.2435 - mean_squared_error: 0.3059 - rmse: 0.4167
step: 237540/583123 epoch 20: ...  - loss: 0.3229 - mean_absolute_error: 0.2468 - mean_squared_error: 0.3140 - rmse: 0.4228
step: 287540/583123 epoch 20: ...  - loss: 0.3194 - mean_absolute_error: 0.2630 - mean_squared_error: 0.3102 - rmse: 0.4319
step: 337540/583123 epoch 20: ...  - loss: 0.3014 - mean_absolute_error: 0.2520 - mean_squared_error: 0.2919 - rmse: 0.4171
step: 387540/583123 epoch 20: ...  - loss: 0.3061 - mean_absolute_error: 0.2458 - mean_squared_error: 0.2969 - rmse: 0.41

Test loss: 0.24154194670862872
val_mean_absolute_error: 0.20695933225676527
score
[0.24154194670862872, 0.20695933225676527, 0.23217169452717262, 0.4216520327460986]
model.metrics_names
['loss', 'mean_absolute_error', 'mean_squared_error', 'rmse']


In [ ]:
0**1

In [ ]:
tf_session = K.get_session()
with tf_session.as_default():
    display(((K.abs(0.)) ** (K.clip(K.abs(.5), 0.5, 3))).eval())

In [ ]:
K.gradients?